In [37]:
%pip install pmdarima

     ------------------------------------ 572.0/572.0 kB 642.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [209]:
import mediapipe as mp
import cv2
import numpy as np
import math
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
from datetime import datetime
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split 
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from statsmodels.tsa.seasonal import seasonal_decompose
from datetime import timedelta
from statsmodels.tsa.arima.model import ARIMA

In [2]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [165]:
data = {"dates" : [],
        "values" : [],
        "spike/deep": []
        }

cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    counter = 0
    iter = 0
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Extract landmarks

        landmarks = results.pose_landmarks.landmark



        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )


        # Get coordinates
        R_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
        R_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
        R_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
        #Visibility Extraction
        R_V = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
        

        L_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        L_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        L_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
        #Visibility Extraction
        L_V = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
               
        if R_V > L_V:
            angle = calculate_angle(R_hip, R_knee, R_ankle)
        else:
            angle = calculate_angle(L_hip, L_knee, L_ankle)
        counter+=1

                    # Visualize angle
        cv2.putText(image, str(angle), 
                        #tuple(np.multiply(L_knee, [1280, 720]).astype(int)), 
                        tuple(np.multiply(L_knee, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA

                            )
        if counter == 4:
            counter = 0
            iter += 1

        """ 
        c = 0
        avg_y = 0
        for i in range(11):
            if landmarks[i+11].visibility > 0.9:
                avg_y += landmarks[i+11].y
                c += 1
        avg_y / c
        """

        avg_y = (landmarks[11].y + landmarks[13].y + landmarks[15].y)/3

        if angle > 160:
            data["spike/deep"].append(1)
            data["values"].append(1-avg_y)
            data["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
        elif angle < 110:
            data["spike/deep"].append(-1)
            data["values"].append(1-avg_y)
            data["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
        else:
            data["spike/deep"].append(0)
            data["values"].append(1-avg_y)
            data["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
            
                
        
        cv2.imshow('Mediapipe Feed', image)
        
        if (cv2.waitKey(10) & 0xFF == ord('q') | (iter == 100)):
            break

    cap.release()
    cv2.destroyAllWindows()

In [166]:
sum = 0
for i in range(len(data)-1):
    d1 = datetime.strptime(data["dates"][i], "%H:%M:%S:%f")
    d2 = datetime.strptime(data["dates"][i+1], "%H:%M:%S:%f")
    delta = d2 - d1
    delta_ms = round(delta.microseconds,0) / 1000
    sum = sum + delta_ms
avg_ms = sum // len(data)-1
print(avg_ms)

47.0


In [167]:
for i in range(100):
    #index = 121 - i
    d1 = datetime.strptime(data["dates"][-1], "%H:%M:%S:%f")
    sum = d1 + timedelta(milliseconds=avg_ms)
    data["dates"].append(datetime.strftime(sum,"%H:%M:%S:%f"))
    data["values"].append(0)
    data["spike/deep"].append(0)
    #data["values"].append(data["values"][len(data)-index:][0])
    #data["spike/deep"].append(data["spike/deep"][len(data)-index:][0])

In [168]:
#To Dataframe
df = pd.DataFrame(data = data)

#df_raw = df_raw.drop(index=range(28))

#without Outliers
#df = df_raw[(df_raw['values'] > df_raw['values'].quantile(q=0.75)) |
#            (df_raw['values'] < df_raw['values'].quantile(q=0.25)) ]

In [169]:
df.to_csv("Squats_Unfiltered.csv" , index = False)

In [198]:
%matplotlib qt
df.plot(kind='line', y = "values", x = "dates", style='-o')

<AxesSubplot:xlabel='dates'>

In [171]:
res = seasonal_decompose(df["values"][:len(df)-100],model = "additive",period=21)
res.plot();

In [158]:
############## NO SPIKES / DEEPS ##################
auto_arima(df["values"][:len(df)-100], m=21,trace=True, seasonal=True).summary()

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,0,1)[21] intercept   : AIC=-3340.973, Time=2.86 sec
 ARIMA(0,0,0)(0,0,0)[21] intercept   : AIC=-970.869, Time=0.10 sec
 ARIMA(1,0,0)(1,0,0)[21] intercept   : AIC=inf, Time=0.86 sec
 ARIMA(0,0,1)(0,0,1)[21] intercept   : AIC=-1787.728, Time=0.79 sec
 ARIMA(0,0,0)(0,0,0)[21]             : AIC=192.497, Time=0.06 sec
 ARIMA(2,0,2)(0,0,1)[21] intercept   : AIC=-3361.812, Time=2.23 sec
 ARIMA(2,0,2)(0,0,0)[21] intercept   : AIC=-3341.220, Time=0.92 sec
 ARIMA(2,0,2)(0,0,2)[21] intercept   : AIC=-3365.128, Time=9.63 sec
 ARIMA(2,0,2)(1,0,2)[21] intercept   : AIC=-3365.331, Time=12.09 sec
 ARIMA(2,0,2)(2,0,2)[21] intercept   : AIC=inf, Time=15.47 sec
 ARIMA(2,0,2)(2,0,1)[21] intercept   : AIC=-3362.992, Time=9.72 sec
 ARIMA(1,0,2)(1,0,2)[21] intercept   : AIC=-3391.799, Time=9.46 sec
 ARIMA(1,0,2)(0,0,2)[21] intercept   : AIC=-3390.975, Time=9.26 sec
 ARIMA(1,0,2)(1,0,1)[21] intercept   : AIC=-3370.643, Time=1.80 sec
 ARIMA(1,0,2)(2,0,

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  570
Model:               SARIMAX(3, 0, 3)   Log Likelihood                1810.982
Date:                Thu, 09 Mar 2023   AIC                          -3607.964
Time:                        18:07:54   BIC                          -3577.545
Sample:                             0   HQIC                         -3596.095
                                - 570                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          2.7575      0.045     61.839      0.000       2.670       2.845
ar.L2         -2.5791      0.086    -30.068      0.000      -2.747      -2.411
ar.L3          0.8212      0.042     19.725      0.000       0.740       0.903
ma.L1         -1.5746      0.063    -24.880      0.000      -1.699      -1.451
ma.L2          0.7102      0.098      7.280      0.000       0.519       0.901
ma.L3          0.0782      0.054      1.443      0.149      -0.028       0.184
sigma2         0.0001   4.43e-06     22.688      0.000    9.18e-05       0.000
===================================================================================
Ljung-Box (L1) (Q):                   0.06   Jarque-Bera (JB):              1009.29
Prob(Q):                              0.81   Prob(JB):                         0.00
Heteroskedasticity (H):               0.31   Skew:                            -0.44
Prob(H) (two-sided):                  0.00   Kurtosis:                         9.46
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
"""for i in range(100):
    index = 100 - i
    if data["values"][len(df)-index:][0] == 0:
       data["values"][len(df)-index:][0] = data["values"][len(df)-index-17:][0]
       print(data["values"][len(df)-index-17:][0])
"""

In [96]:
df.to_csv("1.csv")

In [172]:
############## NO SPIKES / DEEPS ##################
train_df = df.iloc[:len(df)-100]
test_df = df.iloc[len(df)-100:]
model = SARIMAX(train_df["values"], order=(70, 1, 1))
res = model.fit()
start = len(train_df)
end = len(train_df) + len(test_df) -1
prediction = res.predict(start, end).rename("prediction")
ax = df["values"].plot(legend=True,figsize=(16,8))
prediction.plot(legend=True)


c:\Users\mosac\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<AxesSubplot:>

In [329]:
############## NO SPIKES / DEEPS ##################
from statsmodels.tools.eval_measures import rmse
accuracy_no_deeps = 1- rmse(test_df["values"], prediction)
print(accuracy_no_deeps)

0.6427576846799165


In [269]:
############## WITH SPIKES / DEEPS ##################
auto_arima(df["values"][:len(df)-100], exogenous=df["spike/deep"], seasonal=True , start_q=0,max_q= 1, trace  =True).summary()

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=-1769.123, Time=0.59 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=-567.943, Time=0.07 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=-1695.878, Time=0.05 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=-1012.066, Time=0.27 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=331.628, Time=0.03 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=-1781.975, Time=0.34 sec
 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=-1295.928, Time=0.20 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=-1770.561, Time=0.28 sec
 ARIMA(1,0,3)(0,0,0)[0] intercept   : AIC=-1823.179, Time=0.38 sec
 ARIMA(0,0,3)(0,0,0)[0] intercept   : AIC=inf, Time=0.45 sec
 ARIMA(2,0,3)(0,0,0)[0] intercept   : AIC=inf, Time=1.01 sec
 ARIMA(1,0,4)(0,0,0)[0] intercept   : AIC=-1857.999, Time=0.52 sec
 ARIMA(0,0,4)(0,0,0)[0] intercept   : AIC=-1575.606, Time=0.50 sec
 ARIMA(2,0,4)(0,0,0)[0] intercept   : AIC=inf, Time=1.16 sec
 ARIMA(1,0,5)(0,0,0)[0] intercept   : AI

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  375
Model:               SARIMAX(1, 0, 5)   Log Likelihood                 942.525
Date:                Thu, 09 Mar 2023   AIC                          -1869.049
Time:                        14:50:20   BIC                          -1837.634
Sample:                             0   HQIC                         -1856.577
                                - 375                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0149      0.013      1.175      0.240      -0.010       0.040
ar.L1          0.9559      0.032     30.154      0.000       0.894       1.018
ma.L1          0.2676      0.108      2.488      0.013       0.057       0.478
ma.L2          0.2343      0.067      3.495      0.000       0.103       0.366
ma.L3          0.3576      0.048      7.522      0.000       0.264       0.451
ma.L4          0.3231      0.062      5.223      0.000       0.202       0.444
ma.L5          0.1514      0.085      1.786      0.074      -0.015       0.318
sigma2         0.0004   1.24e-05     30.066      0.000       0.000       0.000
===================================================================================
Ljung-Box (L1) (Q):                   0.46   Jarque-Bera (JB):              9920.20
Prob(Q):                              0.50   Prob(JB):                         0.00
Heteroskedasticity (H):               0.80   Skew:                             2.74
Prob(H) (two-sided):                  0.21   Kurtosis:                        27.60
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [377]:
############## WITH SPIKES / DEEPS ##################
train_df = df.iloc[:len(df)-100]
test_df = df.iloc[len(df)-100:] 
model = SARIMAX(endog=train_df["values"], order=(30,1, 0), exog=train_df["spike/deep"])
res = model.fit()
start = len(train_df)
end = len(train_df) + len(test_df) -1
prediction = res.predict(start, end,exog = test_df["spike/deep"]).rename("prediction")
ax = df["values"].plot(legend=True,figsize=(16,8))
prediction.plot(legend=True)

c:\Users\mosac\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<AxesSubplot:>

In [ ]:
############## WITH SPIKES / DEEPS ##################

# Need to make sure to truncate it in the right place
#prediction

In [99]:
############## WITH SPIKES / DEEPS ##################
accuracy = 1- rmse(test_df["values"], prediction)
print(accuracy)

0.9173200783498131


In [75]:
# Old Version Of "calculate_decay_rate()"
def calculate_decay_rate(df):
    decay = []
    #average decay rate
    for i in range(1,len(df) -1 ):
        x1 = df["spike/deep"][i]
        x2 = df["spike/deep"][i+1]
        if (x1 == 1) and (x2 == 0):
            x1_index = i
        elif (x1 == 0)  and (x2 == 1):
            decay.append(x1_index/i)
    decay.append(1)
    return decay


In [203]:
def insert_missing_rows(df,lead):
    for i in range(lead):
        d = datetime.strptime(df["dates"][-1], "%H:%M:%S:%f")
        sum = d + timedelta(seconds=1)
        df["dates"].append(datetime.strftime(sum,"%H:%M:%S:%f"))
        df["values"].append(0)
    return df


In [204]:
#New Versopn Of calculate_decay_rate()
def calculate_decay_rate(df,lead):
    decay = {"dates" : [],
            "values" : []
        }
    #average decay rate
    for i in range(1,len(df) -1 ):
        x1 = df["spike/deep"][i]
        x2 = df["spike/deep"][i+1]
        if (x1 == 1) and (x2 == 0):
            x1_index = i
        elif (x1 == 0)  and (x2 == 1):
            if len(decay["dates"]) == 0:
                decay["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
                decay["values"].append(x1_index/i)
            else:
                d = datetime.strptime(decay["dates"][-1], "%H:%M:%S:%f")
                sum = d + timedelta(seconds=1)
                decay["dates"].append(datetime.strftime(sum,"%H:%M:%S:%f"))
                decay["values"].append(x1_index/i)


    for i in range(lead):
        d = datetime.strptime(decay["dates"][-1], "%H:%M:%S:%f")
        sum = d + timedelta(seconds=1)
        decay["dates"].append(datetime.strftime(sum,"%H:%M:%S:%f"))
        decay["values"].append(0)

    decay = insert_missing_rows(decay,lead)


    return decay


In [206]:
def add_sec(date):
    d = datetime.strptime(date, "%H:%M:%S:%f")
    return d + timedelta(seconds=1)


In [229]:
def add_points(df):
    new = {"dates" : [],
            "values" : []
        }
    for i in range(len(df)-1):
        x1 = df[i]    #left
        x2 = df[i+1]  #right
        print(df)
        m = (x1+x2)/2   #middle
        lm = (x1+m)/2   #left_middle
        rm = (m+x2)/2   #right_middle
        if len(new["dates"]) == 0:
            new["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
            new["values"].append(x1)
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(lm)
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(m)
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(rm)
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(x2)
        else:
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(lm)
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(m)
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(rm)
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(x2)
    return new

            

In [230]:
lead = 5
res = calculate_decay_rate(df,lead)
res_df = pd.DataFrame(data = res)
#ax = res_df[1:].plot(legend=True,figsize=(16,8))
#ax.set_xlim([1,len(res_df)-lead])
new = add_points(res_df["values"])
new
#new_res_df = pd.DataFrame(data = new)



0     0.354839
1     0.644068
2     0.693878
3     0.694268
4     0.787330
5     0.718935
6     0.000000
7     0.000000
8     0.000000
9     0.000000
10    0.000000
11    0.000000
12    0.000000
13    0.000000
14    0.000000
15    0.000000
Name: values, dtype: float64
0     0.354839
1     0.644068
2     0.693878
3     0.694268
4     0.787330
5     0.718935
6     0.000000
7     0.000000
8     0.000000
9     0.000000
10    0.000000
11    0.000000
12    0.000000
13    0.000000
14    0.000000
15    0.000000
Name: values, dtype: float64
0     0.354839
1     0.644068
2     0.693878
3     0.694268
4     0.787330
5     0.718935
6     0.000000
7     0.000000
8     0.000000
9     0.000000
10    0.000000
11    0.000000
12    0.000000
13    0.000000
14    0.000000
15    0.000000
Name: values, dtype: float64
0     0.354839
1     0.644068
2     0.693878
3     0.694268
4     0.787330
5     0.718935
6     0.000000
7     0.000000
8     0.000000
9     0.000000
10    0.000000
11    0.000000
12    0.00000

{'dates': ['15:19:31:407894',
  '15:19:32:407894',
  '15:19:33:407894',
  '15:19:34:407894',
  '15:19:35:407894',
  '15:19:36:407894',
  '15:19:37:407894',
  '15:19:38:407894',
  '15:19:39:407894',
  '15:19:40:407894',
  '15:19:41:407894',
  '15:19:42:407894',
  '15:19:43:407894',
  '15:19:44:407894',
  '15:19:45:407894',
  '15:19:46:407894',
  '15:19:47:407894',
  '15:19:48:407894',
  '15:19:49:407894',
  '15:19:50:407894',
  '15:19:51:407894',
  '15:19:52:407894',
  '15:19:53:407894',
  '15:19:54:407894',
  '15:19:55:407894',
  '15:19:56:407894',
  '15:19:57:407894',
  '15:19:58:407894',
  '15:19:59:407894',
  '15:20:00:407894',
  '15:20:01:407894',
  '15:20:02:407894',
  '15:20:03:407894',
  '15:20:04:407894',
  '15:20:05:407894',
  '15:20:06:407894',
  '15:20:07:407894',
  '15:20:08:407894',
  '15:20:09:407894',
  '15:20:10:407894',
  '15:20:11:407894',
  '15:20:12:407894',
  '15:20:13:407894',
  '15:20:14:407894',
  '15:20:15:407894',
  '15:20:16:407894',
  '15:20:17:407894',
  '1

In [185]:
#Auto Arima To the dacay list
train = res_df[1:-lead]
test = res_df[-lead:]

trend = []
for i in range(lead -1):
    trend.append(lead -1 -i)

auto_arima(train["values"], exogenous = trend,trend="t").summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                    5
Model:               SARIMAX(1, 0, 1)   Log Likelihood                   8.479
Date:                Fri, 10 Mar 2023   AIC                             -8.957
Time:                        14:18:46   BIC                            -10.520
Sample:                             0   HQIC                           -13.150
                                  - 5                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
drift          0.0120      0.009      1.402      0.161      -0.005       0.029
ar.L1          0.9824      0.013     73.612      0.000       0.956       1.009
ma.L1         -0.8693      0.336     -2.584      0.010      -1.529      -0.210
sigma2         0.0018      0.002      0.920      0.358      -0.002       0.006
===================================================================================
Ljung-Box (L1) (Q):                   1.68   Jarque-Bera (JB):                 0.40
Prob(Q):                              0.20   Prob(JB):                         0.82
Heteroskedasticity (H):               4.12   Skew:                             0.41
Prob(H) (two-sided):                  0.39   Kurtosis:                         1.89
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [202]:
from matplotlib import pyplot

model = SARIMAX(endog=train["values"], order=(1,1,0))
model_fit = model.fit()

start = len(train["values"])
end = len(train["values"]) + lead
prediction = model_fit.predict(start, end,exog=trend).rename("prediction")

ax = res_df["values"][1:].plot(legend=True,figsize=(16,8))
prediction.plot(legend=True)

"""
# summary of fit model
print(model_fit.summary())
# line plot of residuals
residuals = pd.DataFrame(model_fit.resid)
residuals.plot()
pyplot.show()
# density plot of residuals
residuals.plot(kind='kde')
pyplot.show()
# summary stats of residuals
print(residuals.describe())
"""

c:\Users\mosac\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


"\n# summary of fit model\nprint(model_fit.summary())\n# line plot of residuals\nresiduals = pd.DataFrame(model_fit.resid)\nresiduals.plot()\npyplot.show()\n# density plot of residuals\nresiduals.plot(kind='kde')\npyplot.show()\n# summary stats of residuals\nprint(residuals.describe())\n"

In [141]:
prediction

14    0.987710
15    0.966148
16    0.973124
17    0.970709
18    0.973554
19    0.969304
20    0.972867
Name: prediction, dtype: float64